<a id='home'></a>
### purpose

* I need range-wide climate data to fit trained gradient forest models to current/future climate

* transform downloaded AdaptWest/ClimateNA data (Tongli et al. 2016 - NOT 2021!!) gridded climate data from Lambert conformal to WGS84
* then extract climate data within the boundaries of the updated species range (see ../02_update_species_range_shapefiles.ipynb)

### outline
1. [create directory structure for new files](#create)

2. [unzip AdaptWest NA_NORM_1961-1990 7z files into netCDF files for each env](#unzip)

3. [convert netCDF files to WGS84](#convert)

4. [crop each netCDF file to each species' shapefile](#crop)

- then convert to dataframe and save as .txt file

5. [combine each species file into a single dataframe](#combine)

- combine

6. [save combined files](#save)

### Notes

- previously** I converted netCDF climate files from AdaptWest to WGS84 and then clipped them to the canonical range shapefiles
- but since I've added polygons (../02_update_species_range_shapefiles.ipynb) I need to re-clip these files
- to preserve previous code, I've made a new directory (`netdir`) with symlinks to the .7z files downloaded from AdaptWest

** notebooks/002_doug_fir_gea/10_maps/clip_climate_data_create_climate_maps.ipynb

** notebooks/007_JP_gea/07_gradient_forest/01_prep_spatial_envdata.ipynb

In [1]:
from pythonimports import *
import geopandas, rioxarray, cartopy.crs, xarray
from shapely.geometry import box, mapping

lview,dview = get_client()

latest_commit()
sinfo(html=True)

56 56
##################################################################
Today:	June 20, 2023 - 07:17:33
python version: 3.8.5
conda env: newpy385

Current commit of pythonimports:
commit 03d76f7a992130f4b94ac34a09ad439e918d3044  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Jun 9 09:42:21 2023 -0400
##################################################################



<a id='create'></a>
# 1. create new directory for clipped files and symlink 7z files

[top](#home)

In [2]:
# where the AdaptWest files are
olddir = '/data/projects/pool_seq/environemental_data/netCDF_files'
netdir = makedir('/data/projects/pool_seq/environemental_data/new_netCDF_files')
olddirs = fs(olddir, dirs=True)
olddirs

['/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2050s',
 '/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp45_2080s',
 '/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp85_2050s',
 '/data/projects/pool_seq/environemental_data/netCDF_files/NA_ENSEMBLE_rcp85_2080s',
 '/data/projects/pool_seq/environemental_data/netCDF_files/NA_NORM_1961-1990',
 '/data/projects/pool_seq/environemental_data/netCDF_files/NA_Reference_files']

In [3]:
# get the 7zip files
sevenzfiles = []
for d in olddirs:
    files = fs(d, endswith='.7z')
    assert len(files) == 1
    src = files[0]
    if 'Reference' not in src:
        dst = src.replace(olddir, netdir)
    else:
        # the elevation data will unzip to a folder instead of spilling out .nc files (easier later on)
        dst = op.join(netdir, op.basename(src))
    makedir(op.dirname(dst))
    os.symlink(src, dst)
    sevenzfiles.append(dst)
sevenzfiles

['/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_Bioclim_netCDF.7z',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2080s/NA_ENSEMBLE_rcp45_2080s_Bioclim_netCDF.7z',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp85_2050s/NA_ENSEMBLE_rcp85_2050s_Bioclim_netCDF.7z',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp85_2080s/NA_ENSEMBLE_rcp85_2080s_Bioclim_netCDF.7z',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_NORM_1961-1990/NA_NORM_1961-1990_netCDF.7z',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_Reference_files_netCDF.7z']

In [4]:
len(sevenzfiles)

6

<a id='unzip'></a>
# 2. unzip 7z files

[top](#home)

In [5]:
# unzip them
for file in sevenzfiles:
    dname = op.dirname(file)
    os.chdir(dname)
    print(ColorText(op.basename(dname)).bold().blue())
    !/data/programs/p7zip_16.02/bin/7z x $file

NA_ENSEMBLE_rcp45_2050s

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,56 CPUs x64)

Scanning the drive for archives:
  0M Scan /data/projects/pool_seq/environem . DF_files/NA_ENSEMBLE_rcp45_2050s                                                                              1 file, 316949670 bytes (303 MiB)

Extracting archive: /data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_Bioclim_netCDF.7z
--
Path = /data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_Bioclim_netCDF.7z
Type = 7z
Physical Size = 316949670
Headers Size = 568
Method = LZMA2:26
Solid = +
Blocks = 2

      0% - AHM.n              1% - AHM.n              2% - AHM.n              3% - AHM.n              3% 1 - bFFP.                4% 1 - bFFP.                5% 1 - bFFP.                6% 1 - bFFP.                7% 1 -

      0% - AHM.n              1% - AHM.n              2% - AHM.n              3% - AHM.n              3% 1 - bFFP.                4% 1 - bFFP.                5% 1 - bFFP.                6% 1 - bFFP.                7% 1 - bFFP.                7% 2 - CMD.n                8% 2 - CMD.n                9% 2 - CMD.n               10% 2 - CMD.n               11% 2 - CMD.n               11% 3 - DD18.               12% 3 - DD18.               13% 3 - DD18.               14% 3 - DD18.               14% 4 - DD5.n               15% 4 - DD5.n               16% 4 - DD5.n               17% 4 - DD5.n               18% 4 - DD5.n               18% 5 - DD_0.               19% 5 - DD_0.               20% 5 - DD_0.               21% 5 - DD_0.               22% 5 - DD_0.               22% 6 - DD_18.n                 23% 6 - DD_18.n                 24% 6 - DD_18.n                 25% 6 - DD_18.n                 25% 7 - eFFP.               26% 7 - eFFP.               27% 7 - eFFP.               28% 7 - eFFP.  

      0% - AHM.n              1% - AHM.n              2% - AHM.n              3% - AHM.n              3% 1 - bFFP.                4% 1 - bFFP.                5% 1 - bFFP.                6% 1 - bFFP.                7% 1 - bFFP.                7% 2 - CMD.n                8% 2 - CMD.n                9% 2 - CMD.n               10% 2 - CMD.n               11% 2 - CMD.n               11% 3 - DD18.               12% 3 - DD18.               13% 3 - DD18.               14% 3 - DD18.               14% 4 - DD5.n               15% 4 - DD5.n               16% 4 - DD5.n               17% 4 - DD5.n               18% 4 - DD5.n               18% 5 - DD_0.               19% 5 - DD_0.               20% 5 - DD_0.               21% 5 - DD_0.               22% 6 - DD_18.n                 23% 6 - DD_18.n                 24% 6 - DD_18.n                 25% 6 - DD_18.n                 25% 7 - eFFP.               26% 7 - eFFP.               27% 7 - eFFP.               28% 7 - eFFP.               29% 8 - EMT.n  

--
Path = /data/projects/pool_seq/environemental_data/new_netCDF_files/NA_Reference_files_netCDF.7z
Type = 7z
Physical Size = 30168790
Headers Size = 257
Method = LZMA2:26
Solid = +
Blocks = 1

      0% - NA_Reference_files_netCDF/ClimateNA_DEM.p                                                  7% 1 - NA_Reference_files_netCDF/ClimateNA_DEM.n                                                   12% 1 - NA_Reference_files_netCDF/ClimateNA_DEM.n                                                   17% 1 - NA_Reference_files_netCDF/ClimateNA_DEM.n                                                   22% 1 - NA_Reference_files_netCDF/ClimateNA_DEM.n                                                   26% 1 - NA_Reference_files_netCDF/ClimateNA_DEM.n                                                   30% 1 - NA_Reference_files_netCDF/ClimateNA_DEM.n                                                   35% 1 - NA_Reference_files_netCDF/ClimateNA_DEM.n                                                   41% 1

<a id='convert'></a>
# 3. convert each netCDF file from Lambert Conformal to WGS84

[top](#home)

In [6]:
# original proj string as-downloaded (the coordinate projection of the data)
# technically `+lon_0=-95.0` was documented as `+lon_0=--95.0` but `--` didn't make any sense
proj = '+proj=lcc +lat_1=49.0 +lat_2=77.0 +lat_0=0.0 +lon_0=-95.0 +x_0=0.0 +y_0=0.0 +ellps=WGS84 \
+datum=WGS84 +units=m +no_defs'

In [7]:
def gdalwarp(netcdf_infile, netcdf_outfile, proj):
    """Convert `netcdf_infile` to WGS84 `netcdf_outfile`.
    
    Notes
    -----
    conda install -c conda-forge gdal 
    """
    import subprocess, shutil
    
    output = subprocess.check_output(['/lu213/brandon.lind/data/anaconda3/envs/gdal_env/bin/gdalwarp',
                                      '-s_srs', proj,
                                      '-t_srs',  '+proj=longlat +ellps=WGS84',
                                      '-of', 'netCDF',
                                      netcdf_infile,
                                      netcdf_outfile,
                                      '-overwrite']).decode('utf-8').split('\n')
    return output

In [8]:
netdirs = fs(netdir, dirs=True)
netdirs

['/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2080s',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp85_2050s',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp85_2080s',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_NORM_1961-1990',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_Reference_files_netCDF']

In [9]:
# get the netCDF files
netfiles = []
for d in netdirs:
    files = fs(d, endswith='.nc', exclude='WGS84')
    print(op.basename(d), len(files))
    netfiles.extend(files)
print(len(netfiles), luni(netfiles))
netfiles[0]

NA_ENSEMBLE_rcp45_2050s 27
NA_ENSEMBLE_rcp45_2080s 27
NA_ENSEMBLE_rcp85_2050s 27
NA_ENSEMBLE_rcp85_2080s 27
NA_NORM_1961-1990 27
NA_Reference_files_netCDF 2
137 137


'/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM.nc'

In [10]:
# test
gdalwarp(netfiles[0], netfiles[0].replace('.nc', '_WGS84.nc'), proj)

['Creating output file that is 15399P x 3236L.',
 'Processing /data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM.nc [1/1] : 0Using internal nodata values (e.g. -3.4e+38) for image /data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM.nc.',
 'Copying nodata values from source /data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM.nc to destination /data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM_WGS84.nc.',
 '...10...20...30...40...50...60...70...80...90...100 - done.',
 '']

In [11]:
# run in parallel
jobs = []
for netfile in netfiles:
    jobs.append(lview.apply_async(gdalwarp, *(netfile, netfile.replace('.nc', '_WGS84.nc'), proj)))
watch_async(jobs, desc='gdalwarp')


Watching 137 jobs ...


gdalwarp: 100%|███████████████| 137/137 [01:32<00:00,  1.48it/s]


In [12]:
# make sure no errors
for j in jobs:
    assert 'Creating' in  j.r[0]

In [13]:
netdirs

['/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2080s',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp85_2050s',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp85_2080s',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_NORM_1961-1990',
 '/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_Reference_files_netCDF']

In [14]:
# double check files were made
outfiles = []
for d in netdirs:
    outfiles.extend(fs(d, 'WGS84', endswith='.nc'))

assert len(outfiles) == len(netfiles)

<a id='crop'></a>
# 4. crop each WGS84 netCDF file using range map for use in future projections

[top](#home)

In [15]:
def clip(netcdf_file, outfile, shapefile):
    """Clip netCDF file within shapefile boundaries, save as tif and dataframe with coords.
    
    Notes
    -----
    - assumes WGS84 projection of netcdf_file
    """
    import geopandas, rioxarray, cartopy.crs, xarray
    from shapely.geometry import box, mapping

    # read in netcdf file
    dataset = xarray.open_dataset(netcdf_file)
    # add metadata for WGS84
    dataset.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    dataset.rio.write_crs("epsg:4326", inplace=True)

    # read in the shapefile
    shp = geopandas.read_file(shapefile, crs=cartopy.crs.PlateCarree())
    
    # clip climate data within boundaries of shapefile
    clipped = dataset.rio.clip(shp.geometry.apply(mapping), shp.crs, drop=False)
    
#     # save as .tif
#     tif = outfile.replace('.txt', '.tif')
#     clipped.rio.to_raster(tif)
    
    # convert to dataframe, remove null points
    df = clipped.to_dataframe()
    df = df[df['Band1'].notnull()]
    # convert multi-index (lat/long) to column data
    df.reset_index(drop=False, inplace=True)
    
    # save
    df.to_csv(outfile, sep='\t', index=False)
    
    return outfile
    

In [16]:
# get the paths to the updated shapefiles
shapedir = '/data/projects/pool_seq/environemental_data/shapefiles'
bnames = ['interior_union_file.shp',
          'combined/df_combined_union_file.shp',
          'jp_union_file.shp',
          'wl_union_file.shp',
          'coastal_union_file.shp']
shapefiles = [op.join(shapedir, f) for f in bnames]
shapefiles

['/data/projects/pool_seq/environemental_data/shapefiles/interior_union_file.shp',
 '/data/projects/pool_seq/environemental_data/shapefiles/combined/df_combined_union_file.shp',
 '/data/projects/pool_seq/environemental_data/shapefiles/jp_union_file.shp',
 '/data/projects/pool_seq/environemental_data/shapefiles/wl_union_file.shp',
 '/data/projects/pool_seq/environemental_data/shapefiles/coastal_union_file.shp']

In [17]:
outfiles[0]

'/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM_WGS84.nc'

In [18]:
for shape in shapefiles:
    print(op.basename(shape).split("_union")[0])

interior
df_combined
jp
wl
coastal


In [19]:
# test
clip(
    outfiles[0],
    outfiles[0].replace('.nc', '_clipped_jp.txt'),
    '/data/projects/pool_seq/environemental_data/shapefiles/jp_union_file.shp'
)

'/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/AHM_WGS84_clipped_jp.txt'

In [20]:
# run in parallel 
jobs = []
for f in outfiles:
    for shapefile in shapefiles:
        spp = op.basename(shapefile).split("_union")[0]
        suffix = '_clipped_' + spp + '.txt'  # _clipped_<species>.txt
        jobs.append(
            lview.apply_async(
                clip, *(f,
                        f.replace('.nc', suffix),
                        shapefile)
            )
        )
watch_async(jobs, desc='clip')


Watching 685 jobs ...


clip: 100%|███████████████| 685/685 [04:13<00:00,  2.70it/s]


In [21]:
# retrieve clipped .txt files and file under directory and species using a dictionary
outs = defaultdict(lambda: defaultdict(list))
for j in jobs:
    f = j.r
    d = op.basename(op.dirname(f))
    spp = op.basename(f).split("_")[-1].rstrip('.txt')
    outs[d][spp].append(f)

for d,sppdict in outs.items():
    print(ColorText(d).bold().blue())
    for spp,files in sppdict.items():
        print(spp, len(files))

NA_ENSEMBLE_rcp45_2050s
interior 27
combined 27
jp 27
wl 27
coastal 27
NA_ENSEMBLE_rcp45_2080s
interior 27
combined 27
jp 27
wl 27
coastal 27
NA_ENSEMBLE_rcp85_2050s
interior 27
combined 27
jp 27
wl 27
coastal 27
NA_ENSEMBLE_rcp85_2080s
interior 27
combined 27
jp 27
wl 27
coastal 27
NA_NORM_1961-1990
interior 27
combined 27
jp 27
wl 27
coastal 27
NA_Reference_files_netCDF
interior 2
combined 2
jp 2
wl 2
coastal 2


In [22]:
def read_file(file):
    import pandas
    import os
    env = os.path.basename(file).split("_WGS84")[0]
    df = pandas.read_table(file)
    df.pop('crs')
    df = df.set_index(['lat','lon'])
    df.columns = [env]
    return df

In [23]:
# test
df1 = read_file(outs['NA_ENSEMBLE_rcp45_2050s']['coastal'][4])
df1.head()

DD_0
lat       lon              
36.251831 -121.776749  14.0
          -121.753370  15.0
          -121.729992  15.0
36.275209 -121.800127  13.0
          -121.776749  22.0

<a id='combine'></a>
# 5. combine each clipped data into file

[top](#home)

In [24]:
# combine txt files into single dataframe
tmp_dfs = defaultdict(dict)
for d, sppdict in outs.items():
    for spp, files in sppdict.items():
        jobs = []
        for f in files:
            jobs.append(lview.apply_async(read_file, f))
        watch_async(jobs, desc=f'{d}-{spp}')
        tmp_dfs[d][spp] = pd.concat([j.r for j in jobs], axis=1)
        display(tmp_dfs[d][spp].head(2))


Watching 27 jobs ...


NA_ENSEMBLE_rcp45_2050s-interior: 100%|███████████████| 27/27 [00:00<00:00, 234464.20it/s]


AHM    CMD   DD18     DD5   DD_0   DD_18   EMT   EXT  \
lat       lon                                                                  
42.002853 -111.724149  26.6  475.0  334.0  2271.0  358.0  3639.0 -29.6  39.5   
          -111.700771  18.9  380.0  207.0  1894.0  454.0  4064.0 -31.3  37.5   

                        Eref    FFP    MAP   MAR  MAT  MCMT    MSP  MWMT  \
lat       lon                                                              
42.002853 -111.724149  896.0  153.0  710.0  17.5  8.9  -2.8  217.0  22.5   
          -111.700771  815.0  136.0  920.0  18.2  7.3  -3.7  240.0  20.6   

                        NFFD    PAS  PPT_sm  PPT_wt    RH    SHM    TD  \
lat       lon                                                            
42.002853 -111.724149  213.0  123.0   108.0   216.0  60.0  103.7  25.4   
          -111.700771  193.0  244.0   115.0   312.0  61.0   85.8  24.3   

                       Tave_sm  Tave_wt   bFFP   eFFP  
lat       lon                                          
42.002853 -111.724149     20.4     -1.7  129.0  283.0  
          -111.700771     18.5     -2.6  143.0  279.0


Watching 27 jobs ...


NA_ENSEMBLE_rcp45_2050s-combined: 100%|███████████████| 27/27 [00:00<00:00, 235929.60it/s]


...

NA_Reference_files_netCDF-wl: 100%|███████████████| 2/2 [00:00<00:00, 32140.26it/s]


ClimateNA_DEM  ClimateNA_ID
lat       lon                                     
43.802969 -119.368800         1419.0    29399748.0
          -119.345422         1430.0    29399750.0


Watching 2 jobs ...


NA_Reference_files_netCDF-coastal: 100%|███████████████| 2/2 [00:00<00:00, 17623.13it/s]


ClimateNA_DEM  ClimateNA_ID
lat       lon                                     
36.251831 -121.776749          354.0    34656812.0
          -121.753370          469.0    34664096.0

In [27]:
# BEFORE REVISION:
# # merge elevation data into each of the files
# dfs = defaultdict(dict)
# for d, sppdict in tmp_dfs.items():
#     if d == 'NA_Reference_files_netCDF':
#         # skip elevation because I don't want to merge elevation into elevation, duh
#         continue
#     for spp,df in sppdict.items():
#         print(ColorText(f'{d}-{spp}').bold().blue())
#         # combine climate and elevation data, use lat/long index to do this properly
#         combo_df = pd.concat([df.copy(), tmp_dfs['NA_Reference_files_netCDF'][spp]], axis=1)
#         combo_df.pop('ClimateNA_ID')  # remove unnecessary info
#         assert combo_df.columns.values[-1] == 'ClimateNA_DEM'
#         combo_df.columns.values[-1] = 'Elevation'  # I prefer this variable name
#         dfs[d][spp] = combo_df.copy()
#         display(dfs[d][spp].head(2))

# FOR REVISION:
# instead of merging elevation data into each of the files
    # just copy `tmp_dfs` into `dfs`
dfs = defaultdict(dict)
for d, sppdict in tmp_dfs.items():
    if d == 'NA_Reference_files_netCDF':
        # skip elevation because I don't want to merge elevation into elevation, duh
        continue
    for spp, df in sppdict.items():
        print(ColorText(f'{d}-{spp}').bold().blue())
#         # combine climate and elevation data, use lat/long index to do this properly
#         combo_df = pd.concat([df.copy(), tmp_dfs['NA_Reference_files_netCDF'][spp]], axis=1)
#         combo_df.pop('ClimateNA_ID')  # remove unnecessary info
#         assert combo_df.columns.values[-1] == 'ClimateNA_DEM'
#         combo_df.columns.values[-1] = 'Elevation'  # I prefer this variable name
#         dfs[d][spp] = combo_df.copy()

        dfs[d][spp] = df.copy()
        display(dfs[d][spp].head(2))

NA_ENSEMBLE_rcp45_2050s-interior


AHM    CMD   DD18     DD5   DD_0   DD_18   EMT   EXT  \
lat       lon                                                                  
42.002853 -111.724149  26.6  475.0  334.0  2271.0  358.0  3639.0 -29.6  39.5   
          -111.700771  18.9  380.0  207.0  1894.0  454.0  4064.0 -31.3  37.5   

                        Eref    FFP    MAP   MAR  MAT  MCMT    MSP  MWMT  \
lat       lon                                                              
42.002853 -111.724149  896.0  153.0  710.0  17.5  8.9  -2.8  217.0  22.5   
          -111.700771  815.0  136.0  920.0  18.2  7.3  -3.7  240.0  20.6   

                        NFFD    PAS  PPT_sm  PPT_wt    RH    SHM    TD  \
lat       lon                                                            
42.002853 -111.724149  213.0  123.0   108.0   216.0  60.0  103.7  25.4   
          -111.700771  193.0  244.0   115.0   312.0  61.0   85.8  24.3   

                       Tave_sm  Tave_wt   bFFP   eFFP  
lat       lon                                          
42.002853 -111.724149     20.4     -1.7  129.0  283.0  
          -111.700771     18.5     -2.6  143.0  279.0

NA_ENSEMBLE_rcp45_2050s-combined


AHM    CMD   DD18     DD5  DD_0   DD_18  EMT   EXT  \
lat       lon                                                                
34.545227 -120.490951  47.7  791.0  279.0  3804.0  10.0  1276.0 -6.0  39.2   
          -120.467573  50.4  800.0  308.0  3890.0   9.0  1218.0 -4.8  39.3   

                         Eref    FFP    MAP   MAR   MAT  MCMT   MSP  MWMT  \
lat       lon                                                               
34.545227 -120.490951  1179.0  334.0  532.0  21.6  15.4  12.1  21.0  19.2   
          -120.467573  1176.0  346.0  508.0  21.5  15.6  12.3  20.0  19.4   

                        NFFD  PAS  PPT_sm  PPT_wt    RH    SHM   TD  Tave_sm  \
lat       lon                                                                  
34.545227 -120.490951  356.0  1.0     3.0   318.0  61.0  905.1  7.1     18.1   
          -120.467573  359.0  1.0     2.0   303.0  62.0  980.7  7.1     18.3   

                       Tave_wt  bFFP   eFFP  
lat       lon                                
34.545227 -120.490951     12.4  22.0  356.0  
          -120.467573     12.6  14.0  360.0

NA_ENSEMBLE_rcp45_2050s-jp


...

AHM    CMD   DD18     DD5   DD_0   DD_18   EMT   EXT  \
lat       lon                                                                 
41.488534 -87.410886  21.0  174.0  504.0  2839.0  444.0  3376.0 -30.3  39.6   
          -87.387508  21.1  179.0  505.0  2843.0  441.0  3370.0 -30.3  39.6   

                       Eref    FFP    MAP   MAR   MAT  MCMT    MSP  MWMT  \
lat       lon                                                              
41.488534 -87.410886  879.0  168.0  955.0  14.1  10.1  -4.9  488.0  23.3   
          -87.387508  882.0  168.0  953.0  14.1  10.1  -4.9  487.0  23.4   

                       NFFD   PAS  PPT_sm  PPT_wt    RH   SHM    TD  Tave_sm  \
lat       lon                                                                  
41.488534 -87.410886  208.0  74.0   297.0   152.0  64.0  47.8  28.3     22.3   
          -87.387508  207.0  74.0   295.0   152.0  64.0  47.9  28.2     22.3   

                      Tave_wt   bFFP   eFFP  
lat       lon                                
41.488534 -87.410886     -3.4  120.0  288.0  
          -87.387508     -3.4  120.0  288.0

NA_NORM_1961-1990-wl


AHM    CMD  DD18     DD5   DD_0   DD_18   EMT   EXT  \
lat       lon                                                                 
43.802969 -119.368800  45.5  662.0  64.0  1491.0  458.0  4354.0 -34.5  38.5   
          -119.345422  47.6  669.0  66.0  1504.0  461.0  4347.0 -34.5  38.4   

                        Eref   FFP    MAP   MAR  MAT  MCMT    MSP  MWMT  \
lat       lon                                                             
43.802969 -119.368800  877.0  76.0  355.0  18.3  6.2  -3.8  109.0  17.2   
          -119.345422  878.0  77.0  340.0  18.5  6.2  -3.9  107.0  17.3   

                        NFFD   PAS  PPT_sm  PPT_wt    RH    SHM    TD  \
lat       lon                                                           
43.802969 -119.368800  147.0  97.0    59.0   121.0  49.0  158.3  21.0   
          -119.345422  148.0  91.0    58.0   112.0  49.0  161.2  21.2   

                       Tave_sm  Tave_wt   bFFP   eFFP  
lat       lon                                          
43.802969 -119.368800     15.6     -2.6  170.0  245.0  
          -119.345422     15.7     -2.6  169.0  246.0

NA_NORM_1961-1990-coastal


AHM    CMD   DD18     DD5  DD_0   DD_18   EMT   EXT  \
lat       lon                                                                 
36.251831 -121.776749  33.6  678.0  151.0  3050.0  22.0  1901.0 -10.0  36.9   
          -121.753370  28.7  691.0  239.0  3182.0  25.0  1855.0 -10.3  37.7   

                         Eref    FFP    MAP   MAR   MAT  MCMT   MSP  MWMT  \
lat       lon                                                               
36.251831 -121.776749  1057.0  278.0  692.0  18.0  13.3   9.1  26.0  17.8   
          -121.753370  1079.0  277.0  822.0  17.5  13.6   8.8  29.0  19.5   

                        NFFD  PAS  PPT_sm  PPT_wt    RH    SHM    TD  Tave_sm  \
lat       lon                                                                   
36.251831 -121.776749  338.0  3.0     7.0   373.0  63.0  674.8   8.6     17.0   
          -121.753370  337.0  4.0     8.0   445.0  63.0  667.7  10.7     18.4   

                       Tave_wt  bFFP   eFFP  
lat       lon                                
36.251831 -121.776749      9.5  61.0  339.0  
          -121.753370      9.2  63.0  340.0

<a id='save'></a>
# 6. save

[top](#home)

In [28]:
# save
for d, sppdict in dfs.items():
    for spp,df in sppdict.items():
        print(ColorText(f'{d}-{spp}').bold().blue())
        _d = op.join(netdir, d) 
        assert op.exists(_d)
        f = op.join(_d, f'{d}_all-envs_WGS84_clipped_{spp}.txt')
        print(f)
        df.reset_index(drop=False, inplace=True)  # this creates a 'lat' and a 'lon' column in the data
        df.to_csv(f, sep='\t', index=False)

NA_ENSEMBLE_rcp45_2050s-interior
/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_all-envs_WGS84_clipped_interior.txt
NA_ENSEMBLE_rcp45_2050s-combined
/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_all-envs_WGS84_clipped_combined.txt
NA_ENSEMBLE_rcp45_2050s-jp
/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_all-envs_WGS84_clipped_jp.txt
NA_ENSEMBLE_rcp45_2050s-wl
/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_all-envs_WGS84_clipped_wl.txt
NA_ENSEMBLE_rcp45_2050s-coastal
/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2050s/NA_ENSEMBLE_rcp45_2050s_all-envs_WGS84_clipped_coastal.txt
NA_ENSEMBLE_rcp45_2080s-interior
/data/projects/pool_seq/environemental_data/new_netCDF_files/NA_ENSEMBLE_rcp45_2080s/NA_ENSEMBLE_rcp45_

In [29]:
# create README

readme = op.join(netdir, 'README.txt')

with open(readme, 'w') as o:
    text = '''June 20, 2023 - Brandon Lind
The files within these folders were clipped to species' range shapefiles that had
added in polygons to account for populations outside of the canonical rangemaps downloaded online.

The other dir (../netCDF_files) were clipped using the canonical rangemap shapefiles.

The other other dir (../new_netCDF_files_before_revision) has same files as here but added elevation

Files were downloaded in February 2021. The proper reference is Wang et al 2016.
--------------------------
'''
    o.write(text)
    
print(read(readme, lines=False))

June 20, 2023 - Brandon Lind
The files within these folders were clipped to species' range shapefiles that had
added in polygons to account for populations outside of the canonical rangemaps downloaded online.

The other dir (../netCDF_files) were clipped using the canonical rangemap shapefiles.

The other other dir (../new_netCDF_files_before_revision) has same files as here but added elevation

Files were downloaded in February 2021. The proper reference is Wang et al 2016.
--------------------------



[top](#home)